## 准备数据

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [5]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [9]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1 = tf.Variable(tf.random.normal(shape=(28 * 28, 128), stddev=0.01), trainable=True)
        self.b1 = tf.Variable(tf.zeros(shape=(128,)), trainable=True)
        self.W2 = tf.Variable(tf.random.normal(shape=(128, 10), stddev=0.01), trainable=True)
        self.b2 = tf.Variable(tf.zeros(shape=(10,)), trainable=True)
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        x_flattened = tf.reshape(x, [-1, 784])
        h1 = tf.matmul(x_flattened, self.W1) + self.b1
        activated_h1 = tf.tanh(h1)
        logits = tf.matmul(activated_h1, self.W2) + self.b2
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [11]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [13]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.3016756 ; accuracy 0.14213334
epoch 1 : loss 2.3014057 ; accuracy 0.14915
epoch 2 : loss 2.301136 ; accuracy 0.15553333
epoch 3 : loss 2.3008657 ; accuracy 0.16343333
epoch 4 : loss 2.3005953 ; accuracy 0.17116667
epoch 5 : loss 2.3003247 ; accuracy 0.17893334
epoch 6 : loss 2.300054 ; accuracy 0.18646666
epoch 7 : loss 2.299783 ; accuracy 0.19441667
epoch 8 : loss 2.299512 ; accuracy 0.20188333
epoch 9 : loss 2.2992406 ; accuracy 0.20866667
epoch 10 : loss 2.2989688 ; accuracy 0.21653333
epoch 11 : loss 2.2986968 ; accuracy 0.22311667
epoch 12 : loss 2.2984245 ; accuracy 0.22986667
epoch 13 : loss 2.2981517 ; accuracy 0.23633334
epoch 14 : loss 2.2978787 ; accuracy 0.24326667
epoch 15 : loss 2.2976053 ; accuracy 0.24966666
epoch 16 : loss 2.2973313 ; accuracy 0.25556666
epoch 17 : loss 2.297057 ; accuracy 0.26101667
epoch 18 : loss 2.2967823 ; accuracy 0.266
epoch 19 : loss 2.296507 ; accuracy 0.27171665
epoch 20 : loss 2.2962313 ; accuracy 0.27691665
epoch 21 : loss 